In [1]:
import urllib.request
import json
import glob
import os
import boto3
import botocore
import time
from datetime import datetime


from dotenv import load_dotenv
load_dotenv()


awskey = os.getenv('aws_key')
awssecret = os.getenv('aws_secret')
s3_bucket = os.getenv('s3_bucket')

s3 = boto3.resource('s3',
                    region_name="us-east-1",
                    aws_access_key_id=awskey,
                    aws_secret_access_key=awssecret
                    )

bucket = s3.Bucket(s3_bucket)

In [2]:
def save_to_bucket(data, filepath):
    '''
    Save file to bucket
    '''
    s3object = s3.Object(s3_bucket, filepath)
    s3object.put(
        Body=(bytes(json.dumps(data).encode('UTF-8')))
    )

In [8]:
def download_season_schedule(season_to_get):
    '''
    This function takes the
    '''
    print('Getting Season Schedule')
    game_schedule_url = f'https://www.nfl.com/feeds-rs/schedules/{season_to_get}.json'
    with urllib.request.urlopen(game_schedule_url) as url:
        data = json.loads(url.read().decode())
    game_schedule = data.get("gameSchedules", "")
    team_id_list = []
    for game in game_schedule:
        game_week_type = game.get("seasonType", "")
        game_week = game.get("week", "")
        game_id = game.get("gameId", "")
        filepath = f'gameids/{game_id}.json'
        save_to_bucket(game_id, filepath)

        season = game.get("season", "")
        season_type = game.get("seasonType", "")
        week = game.get("week", "")
        game_id = game.get("gameId", "")
        game_key = game.get("gameKey", "")
        home_id = game.get("homeTeamId", "")
        vis_id =game.get("visitorTeamId", "")
        game_type = game.get("gameType", "")
        week_name_abbr = game.get("weekNameAbbr", "")
        week_name = game.get("weekName", "")

        schedule_details ={'season':season, 'season_type':season_type,'week':week,'game_id':game_id,'game_key':game_key,'home_id':home_id,'vis_id':vis_id,'game_type':game_type,
                       'week_name_abbr':week_name_abbr, 'week_name':week_name}

        filepath = filepath = f'schedules_details/{game_id}.json'
        save_to_bucket(schedule_details, filepath)

        if home_id not in team_id_list:
            team_id_list.append(home_id)
        if vis_id not in team_id_list:
            team_id_list.append(vis_id)

    if '8700' in team_id_list:
        team_id_list.remove('8700')
    if '8600' in team_id_list:
        team_id_list.remove('8600')

    game_schedule_url = f'https://www.nfl.com/feeds-rs/teams/{season_to_get}.json'
    with urllib.request.urlopen(game_schedule_url) as url:
        sch_data = json.loads(url.read().decode())
    team_info = sch_data.get("teams", "")
    
    print('Getting Getting Coach Information')
    print('Getting Getting Team Information')
    for team_id in team_id_list:
        coach_url = f'https://www.nfl.com/feeds-rs/coach/byTeam/{team_id}/{season_to_get}.json' 
        with urllib.request.urlopen(coach_url) as url:
            data = json.loads(url.read().decode())

        esbid = data['esbId']
        pic_url = f'http://static.nfl.com/static/content/public/static/img/fantasy/transparent/200x200/{esbid}.png'
        dateobj = datetime.strptime(data['birthDate'], '%m/%d/%Y')
        birthdate = dateobj.strftime("%Y,%m,%d")

        coachjson = {'coach_id': data['nflId'], 'season': season_to_get, 'week': week, 'display_name': data['displayName'], 'first_name': data['firstName'], 'last_name': data['lastName'], 
                         'esbid': data['esbId'],'status': data['currentStatus'], 'birthdate': birthdate, 'hometown': data['homeTown'], 'college': data['collegeName'], 
                         'team_id': team_id, 'isdeceased': data['isDeceased'], 'pic_url': pic_url}

        filepath = f'coaches/{season}_{week}_{team_id}.json'
        save_to_bucket(coachjson, filepath)

        for team in team_info:
            teamidnum = team.get("teamId", "")
            if teamidnum == team_id:
                division_abbr = team.get('divisionAbbr', '')

                teamjson = {'team_id':team_id, 'season':season, 'abbr': team['abbr'], 'citystate':team['cityState'], 'full_name': team['fullName'], 'nick':team['nick'],
                            'team_type': team['teamType'], 'conference_abbr': team['conferenceAbbr'], 'division_abbr': division_abbr, 'year_found':team['yearFound'], 'stadium_name':team['stadiumName']}

                filepath = f'teams/{season}_{team_id}.json'
                save_to_bucket(teamjson, filepath)

    
    
    
    print('Complete')

In [4]:
def main():
    '''
    '''
    season_to_get = input(str('Enter Season to download: '))
    download_season_schedule(season_to_get)

In [11]:
main()

Enter Season to download:  2019


Getting Season Schedule
Getting Getting Coach Information
Getting Getting Team Information
Complete
